## Save all pages as HTML files

In [1]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random

In [ ]:
# site to scrap
source = 'https://www.autoevolution.com/moto'

# url to scrap
url = 'https://www.autoevolution.com/moto'
response = requests.get(url)
file_name = "main_page"

with open(f"pages/{file_name}.html", "w") as file:
    file.write(response.text)
    file.close()
    
soup = BeautifulSoup(response.content, "html.parser")

brand_soup = soup.find_all('div', class_="col2width fl bcol-white carman")
count_brand = 0

for brand in brand_soup[119:]:
    brand_url= brand.find("a").get("href")
    response_brand = requests.get(brand_url)
    brand_url_soup = BeautifulSoup(response_brand.content, "html.parser")
    model_soup = brand_url_soup.find_all('div', class_="col2width bcol-white fl")
  
    print('brand:', count_brand)
    print(brand_url)
    
    count_model = 0

    for model in model_soup:
        model_url= model.find("a").get("href")
        print('model',count_model)
        print(model_url)
        response_model = requests.get(model_url)

        model_url_soup = BeautifulSoup(response_model.content, "html.parser")

        submodel_soup = model_url_soup.find_all('div', class_="container carmodel clearfix")

        for submodel in submodel_soup:
            submodel_url= submodel.find("a").get("href")
            print(submodel_url)
            response_submodel = requests.get(submodel_url)
            submodel_url_soup = BeautifulSoup(response_submodel.content, "html.parser")
            
            try:
                segment = submodel_url_soup.select('p.mgbot_20')[0].text.replace('Segment: ','')
                brand =  submodel_url_soup.select('.motlisthead_prod')[0].text
                title_description =  submodel_url_soup.select('.motlisthead')[0].text
                first_year_prod = submodel_url_soup.select('.motlisthead_years')[0].text.split(' - ')[0]
                last_year_prod = submodel_url_soup.select('.motlisthead_years')[0].text.split(' - ')[1]
                motor_type = submodel_url_soup.select('div.techdata:nth-child(1) > dl:nth-child(2) > dd:nth-child(2)')[0].text
                motor_size = submodel_url_soup.select('div.techdata:nth-child(1) > dl:nth-child(2) > dd:nth-child(4)')[0].text.replace(" cm3",'')
                motor_power = submodel_url_soup.select('div.techdata:nth-child(1) > dl:nth-child(2) > dd:nth-child(10)')[0].text
                fuel_system = submodel_url_soup.select('div.techdata:nth-child(1) > dl:nth-child(2) > dd:nth-child(14)')[0].text
                gearbox = submodel_url_soup.select('div.techdata:nth-child(2) > dl:nth-child(2) > dd:nth-child(2)')[0].text
                clutch = submodel_url_soup.select('div.techdata:nth-child(2) > dl:nth-child(2) > dd:nth-child(4)')[0].text
                primary_drive = submodel_url_soup.select('div.techdata:nth-child(2) > dl:nth-child(2) > dd:nth-child(4)')[0].text
                final_drive = submodel_url_soup.select('div.techdata:nth-child(2) > dl:nth-child(2) > dd:nth-child(8)')[0].text
                length = submodel_url_soup.select('div.techdata:nth-child(4) > dl:nth-child(2) > dd:nth-child(2)')[0].text
                width = submodel_url_soup.select('div.techdata:nth-child(4) > dl:nth-child(2) > dd:nth-child(4)')[0].text
                weight = submodel_url_soup.select('div.techdata:nth-child(4) > dl:nth-child(2) > dd:nth-child(12)')[0].text
                fuel_capacity = submodel_url_soup.select('div.techdata:nth-child(4) > dl:nth-child(2) > dd:nth-child(14)')[0].text
                description = submodel_url_soup.select(".intro")[0].text

                data = {'segment':[segment],
                        'brand ':[brand],
                        'title_description':[title_description],
                        'description': [description],
                        'first_year_prod':[first_year_prod],
                        'last_year_prod':[last_year_prod],
                        'motor_type':[motor_type],
                        'motor_size':[motor_size],
                        'motor_power':[motor_power],
                        'fuel_system':[fuel_system],
                        'gearbox':[gearbox],
                        'clutch':[clutch],
                        'primary_drive':[primary_drive],
                        'final_drive':[final_drive],
                        'length':[length],
                        'width':[width],
                        'weight':[weight],
                        'fuel_capacity':[fuel_capacity],
                        'description':[description]}

                df = pd.DataFrame(data)
                df.to_csv(f"bike/{title_description}-{first_year_prod}-{last_year_prod}.csv")
                time.sleep(1)
            
            except:
                pass
        count_model += 1
    count_brand += 1

In [23]:
history = pd.DataFrame(
    data = {'segment':[],
            'brand ':[],
            'title_description':[],
            'description': [],
            'first_year_prod':[],
            'last_year_prod':[],
            'motor_type':[],
            'motor_size':[],
            'motor_power':[],
            'fuel_system':[],
            'gearbox':[],
            'clutch':[],
            'primary_drive':[],
            'final_drive':[],
            'length':[],
            'width':[],
            'weight':[],
            'fuel_capacity':[],
            'description':[]})

for filename in [file for file in os.listdir('bike') if file.endswith(".csv")]:
    df = pd.read_csv(f"bike/{filename}")
    history = history.append(df)

history.to_csv("master.csv", index=False)

In [32]:
df = pd.read_csv('master.csv')
df.head()

,segment,brand,title_description,description,first_year_prod,last_year_prod,motor_type,motor_size,motor_power,fuel_system,gearbox,clutch,primary_drive,final_drive,length,width,weight,fuel_capacity,bike_model
0,\nSport Touring\t\t\t,TRIUMPH,TRIUMPH Rocket III Touring 2011 - Present,Designed for riders who demand serious long ha...,2011.0,Present,"liquid-cooled, DOHC, in-line 3-cylinder","2,294",106/5750 KW(hp)/RPM,multipoint sequential electronic fuel injectio...,5-speed,"wet, multiplate","wet, multiplate",shaft,102.4 in OR 2601 mm,39.1 in OR 993 mm,869 lbs OR 394 kg,5.9 gallons OR 22.3 L,Rocket III Touring
1,\nTouring\t\t\t,HARLEY DAVIDSON,HARLEY DAVIDSON Electra Glide Classic 2004 - 2005,The 2005 MY Harley Davidson FLHTC Electra Glid...,2004.0,2005,"air-cooled, Twin Cam 88",1450,67/5200 KW(hp)/RPM,Electronic Sequential Port Fuel Injection (ESP...,"5-speed, manual","wet multi-disc, manual","wet multi-disc, manual",belt,98.3 in OR 2497 mm,38 in OR 965 mm,827 lbs OR 375 kg,6 gallons OR 22.7 L,Electra Glide Classic
2,\nCustom/Cruiser\t\t\t,TRIUMPH,TRIUMPH Thunderbird Storm 2010 - 2011,If you thought the 2010 Thunderbird was a toug...,2010.0,2011,"4-stroke liquid-cooled, DOHC, parallel-twin",1699,97/5200 KW(hp)/RPM,multipoint sequential electronic fuel injection,"6-speed, helical type 2nd - 6th","wet, multiplate","wet, multiplate",toothed belt,92.1 in OR 2339 mm,34.6 in OR 879 mm,746 lbs OR 338 kg,5.8 gallons OR 22.0 L,Thunderbird Storm
3,\nSport / Super Sport\t\t\t,BIMOTA,BIMOTA YB8 1991 - 1992,"The 1992 MY Bimota YB8 is a powerful, liter-cl...",1991.0,1992,"liquid-cooled, four-stroke, transverse four cy...",1002,149/10500 KW(hp)/RPM,4x38mm Mikuni carburetors,"5-speed, manual","wet multi-disc, manual","wet multi-disc, manual",chain,-,-,397 lbs OR 180 kg,5.3 gallons OR 20.1 L,YB8
4,\nATV/Quad\t\t\t,HYOSUNG,HYOSUNG WOW 50 2005 - 2006,The 2006 MY Hyosung WOW 50 is a small ATV suit...,2005.0,2006,"air-cooled, two-stroke, single cylinder",49,-/- KW(hp)/RPM,carburetor,CVT,"dry, centrifugal","dry, centrifugal",chain,49.2 in OR 1250 mm,30.7 in OR 780 mm,187 lbs OR 85 kg,1.1 gallons OR 4.2 L,WOW 50


In [25]:
def parse_brand(brand, description):
    return " " .join(" ".join(description.replace(brand + ' ', '').split(' - ')[:-1]).split()[:-1])

In [26]:
df['bike_model'] = df.apply(lambda x: parse_brand(x['brand '],x['title_description']), axis=1)
df['bike_model']

0                         Rocket III Touring
1                      Electra Glide Classic
2                          Thunderbird Storm
3                                        YB8
4                                     WOW 50
                        ...                 
11753                                    848
11754    KingQuad 750AXi Power Steering Camo
11755                        Prairie 360 4x4
11756                  VT1100C2 Shadow Sabre
11757                                 WR 300
Name: bike_model, Length: 11758, dtype: object

In [29]:
df.drop(columns = 'Unnamed: 0', inplace=True)

In [30]:
df.head()

,segment,brand,title_description,description,first_year_prod,last_year_prod,motor_type,motor_size,motor_power,fuel_system,gearbox,clutch,primary_drive,final_drive,length,width,weight,fuel_capacity,bike_model
0,\nSport Touring\t\t\t,TRIUMPH,TRIUMPH Rocket III Touring 2011 - Present,Designed for riders who demand serious long ha...,2011.0,Present,"liquid-cooled, DOHC, in-line 3-cylinder","2,294",106/5750 KW(hp)/RPM,multipoint sequential electronic fuel injectio...,5-speed,"wet, multiplate","wet, multiplate",shaft,102.4 in OR 2601 mm,39.1 in OR 993 mm,869 lbs OR 394 kg,5.9 gallons OR 22.3 L,Rocket III Touring
1,\nTouring\t\t\t,HARLEY DAVIDSON,HARLEY DAVIDSON Electra Glide Classic 2004 - 2005,The 2005 MY Harley Davidson FLHTC Electra Glid...,2004.0,2005,"air-cooled, Twin Cam 88",1450,67/5200 KW(hp)/RPM,Electronic Sequential Port Fuel Injection (ESP...,"5-speed, manual","wet multi-disc, manual","wet multi-disc, manual",belt,98.3 in OR 2497 mm,38 in OR 965 mm,827 lbs OR 375 kg,6 gallons OR 22.7 L,Electra Glide Classic
2,\nCustom/Cruiser\t\t\t,TRIUMPH,TRIUMPH Thunderbird Storm 2010 - 2011,If you thought the 2010 Thunderbird was a toug...,2010.0,2011,"4-stroke liquid-cooled, DOHC, parallel-twin",1699,97/5200 KW(hp)/RPM,multipoint sequential electronic fuel injection,"6-speed, helical type 2nd - 6th","wet, multiplate","wet, multiplate",toothed belt,92.1 in OR 2339 mm,34.6 in OR 879 mm,746 lbs OR 338 kg,5.8 gallons OR 22.0 L,Thunderbird Storm
3,\nSport / Super Sport\t\t\t,BIMOTA,BIMOTA YB8 1991 - 1992,"The 1992 MY Bimota YB8 is a powerful, liter-cl...",1991.0,1992,"liquid-cooled, four-stroke, transverse four cy...",1002,149/10500 KW(hp)/RPM,4x38mm Mikuni carburetors,"5-speed, manual","wet multi-disc, manual","wet multi-disc, manual",chain,-,-,397 lbs OR 180 kg,5.3 gallons OR 20.1 L,YB8
4,\nATV/Quad\t\t\t,HYOSUNG,HYOSUNG WOW 50 2005 - 2006,The 2006 MY Hyosung WOW 50 is a small ATV suit...,2005.0,2006,"air-cooled, two-stroke, single cylinder",49,-/- KW(hp)/RPM,carburetor,CVT,"dry, centrifugal","dry, centrifugal",chain,49.2 in OR 1250 mm,30.7 in OR 780 mm,187 lbs OR 85 kg,1.1 gallons OR 4.2 L,WOW 50


In [31]:
df.to_csv('master.csv', index=False)